In [94]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [95]:
import numpy as np
import pandas as pd

In [96]:
import os

for root, dirs, files in os.walk("/kaggle/input"):
    for name in files:
        #print(os.path.join(root, name))
        pass

In [97]:
import os
base_dir='/kaggle/working/data'
train_cats=os.path.join(base_dir,"cats")
train_dogs=os.path.join(base_dir,"dogs")
os.makedirs(train_cats,exist_ok=True)
os.makedirs(train_dogs,exist_ok=True)

In [98]:
import shutil
src_dir='/kaggle/input/oscar-dog/train'
for file in os.listdir(src_dir):
    src_path=os.path.join(src_dir,file)
    if file.startswith("cat"):
        shutil.copy(src_path,train_cats)
    elif file.startswith("dog"):
        shutil.copy(src_path,train_dogs)
    

In [99]:
print("Cats:",len(os.listdir(train_cats)))
print("Dogs:",len(os.listdir(train_dogs)))

Cats: 12500
Dogs: 12500


In [100]:
import os as os
base_dir='/kaggle/working/dataset'
for split in['train','val']:
    for cls in ['dog','cat']:
        os.makedirs(os.path.join(base_dir,split,cls),exist_ok=True)

In [101]:
import random
def split_data(src_dir,train_dir,val_dir,split_ratio=0.8):
    files=os.listdir(src_dir)
    random.shuffle(files)
    split=int(len(files)*0.8)
    for f in files[:split]:
        shutil.copy(os.path.join(src_dir,f),train_dir)
    for f in files[split:]:
        shutil.copy(os.path.join(src_dir,f),val_dir)

In [102]:
split_data("/kaggle/working/data/cats","/kaggle/working/dataset/train/cat","/kaggle/working/dataset/val/cat")

In [103]:
split_data("/kaggle/working/data/dogs","/kaggle/working/dataset/train/dog","/kaggle/working/dataset/val/dog")

In [104]:
for split in ["train", "val"]:
    print(f"\n{split.upper()}")
    for cls in os.listdir(f"/kaggle/working/dataset/{split}"):
        print(cls, ":", len(os.listdir(f"/kaggle/working/dataset/{split}/{cls}")))


TRAIN
dog : 12397
cat : 12400

VAL
dog : 6078
cat : 6102


In [105]:
import tensorflow as tf
from tensorflow import keras
img_size=(224,224)
batch_size=32
AUTOTUNE=tf.data.AUTOTUNE
train_ds=keras.utils.image_dataset_from_directory('/kaggle/working/dataset/train',image_size=img_size,
                                                 batch_size=batch_size,shuffle=True)
val_ds=keras.utils.image_dataset_from_directory('/kaggle/working/dataset/val',image_size=img_size,
                                                 batch_size=batch_size,shuffle=True)


Found 24797 files belonging to 2 classes.
Found 12180 files belonging to 2 classes.


In [106]:
train_ds=train_ds.prefetch(AUTOTUNE)
val_ds=val_ds.prefetch(AUTOTUNE)

In [107]:
import tensorflow
from tensorflow.keras.applications import VGG16

In [108]:
base_model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [109]:
for layers in base_model.layers:
    base_model.trainable=False

In [110]:
from tensorflow.keras.layers import *

In [111]:
from tensorflow.keras.models import Sequential

In [118]:
model=Sequential()
model.add(Rescaling(1./255.,input_shape=(224,224,3)))
model.add(RandomRotation(0.2))
model.add(RandomFlip(0.2))
model.add(RandomZoom(0.1))
model.add(RandomTranslation(width_factor=0.2,height_factor=0.2))

model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128))
model.add(Activation('relu'))


model.add(Dense(64))

model.add(Activation('relu'))


model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(1,activation='sigmoid'))

In [119]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-4),loss='binary_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_8               │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_8 (RandomFlip)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_8 (RandomZoom)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation_6            │ (None, 224, 224, 3)    │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,790,721 (56.42 MB)

 Trainable params: 76,033 (297.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [120]:
history=model.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3,min_delta=0.001,restore_best_weights=True)])

Epoch 1/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 179s 228ms/step - accuracy: 0.7098 - loss: 0.5642 - val_accuracy: 0.8956 - val_loss: 0.2504
Epoch 2/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 170s 219ms/step - accuracy: 0.8364 - loss: 0.3681 - val_accuracy: 0.9080 - val_loss: 0.2218
Epoch 3/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 170s 219ms/step - accuracy: 0.8472 - loss: 0.3455 - val_accuracy: 0.9165 - val_loss: 0.1994
Epoch 4/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 170s 220ms/step - accuracy: 0.8530 - loss: 0.3342 - val_accuracy: 0.9207 - val_loss: 0.1915
Epoch 5/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 169s 219ms/step - accuracy: 0.8551 - loss: 0.3278 - val_accuracy: 0.9190 - val_loss: 0.1922
Epoch 6/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 170s 220ms/step - accuracy: 0.8545 - loss: 0.3202 - val_accuracy: 0.9240 - val_loss: 0.1844
Epoch 7/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 169s 219ms/step - accuracy: 0.8560 - loss: 0.3157 - val_accuracy: 0.9221 - val_loss: 0.1856
Epoch 8/20
775/775 ━━━━━━━━━━━━━━━━━━━━ 170s 220ms/step - accuracy: 0.8628 -

In [121]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")

381/381 ━━━━━━━━━━━━━━━━━━━━ 56s 147ms/step - accuracy: 0.9199 - loss: 0.1889
Validation Loss: 0.1844
Validation Accuracy: 0.9240


In [128]:
def predict_image(img):
    import tensorflow as tf
    import numpy as np

    IMG_SIZE = 224

    # Safety check 1: None input
    if img is None:
        return "No image provided"

    # Safety check 2: Ensure PIL → numpy
    if not isinstance(img, np.ndarray):
        img = np.array(img)

    # Safety check 3: Grayscale image → convert to RGB
    if len(img.shape) == 2:  # (H, W)
        img = np.stack([img] * 3, axis=-1)

    # Safety check 4: RGBA → RGB
    if img.shape[-1] == 4:
        img = img[..., :3]

    # Resize
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))

    # Add batch dimension
    img = tf.expand_dims(img, axis=0)

    # Predict
    prob = model.predict(img)[0][0]

    # Safe decoding
    label = "Dog" if prob >= 0.5 else "Cat"

    return f"{label} (confidence: {prob:.3f})"

In [131]:
from PIL import Image
img=Image.open('/kaggle/input/dogcat/cat.jpeg')

In [132]:
predict_image(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


'Cat (confidence: 0.002)'

In [133]:
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Textbox(label="Prediction"),
    title="Cat vs Dog Classifier (CNN from Scratch)",
    description="CNN trained from scratch with transfer learning using VGG16 ."
)

In [ ]:
interface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://b3e2463a773189027b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
